In [1]:
import sys
import numpy as np
import pandas as pd
import csv
import math

In [2]:
data = []
for i in range(18):
    data.append([])
text = open('ml2019spring-hw1/train.csv', 'r', encoding = 'big5')

n_row = 0

row = csv.reader(text, delimiter = ',')
for r in row:
    if n_row != 0:
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))
            
    n_row += 1
text.close()

In [3]:
month_dict = {}

for month in range(12):
    template = np.empty(shape = (18,480))
    for element in range(18):
#         if element != 0 and element != 10 and element != 16 and element != 17:
        for hour in range(480):
            template[element][hour] = data[element][hour+(month*480)]
    month_dict[month] = template

In [4]:
x = np.empty(shape = (471*12, 13*9))
y = np.empty(shape = (471*12, 1))

for month in range(12):
    for hour in range(471):
        n_element = 0
        for element in range(18):
            if element != 0 and element != 10 and element != 16 and element != 17 and element != 4:#4,14, 15
                for h in range(9):
                    x[hour+(month*471)][(n_element*9)+h] = month_dict[month][element][hour+h]
                n_element += 1
        y[month*471 + hour] = month_dict[month][9][9+hour]
print(x.shape)
print(y.shape)

(5652, 117)
(5652, 1)


In [6]:
df = pd.DataFrame()
n_data = 0
for i in range(len(x[0])):
    df[i] = x[:,i]
    n_data += 1

df[n_data] = y

# stats.pearsonr(a,y)
# coeff = df.corr()[117]
# for i in range(len(coeff)):
#     if i % 9 == 0:
#         print('--------',(i/9),'---------')
#     print(coeff[i])

In [7]:
mean = np.mean(x, axis = 0) 
std = np.std(x, axis = 0)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        if not std[j] == 0 :
            x[i][j] = (x[i][j]- mean[j]) / std[j]

In [8]:
x = np.concatenate((np.ones((x.shape[0], 1 )), x) , axis = 1)

In [9]:
w = np.zeros(shape = (len(x[0]), 1 ))
l_rate = 1
repeat = 200000

In [10]:
x_t = np.transpose(x)

beta1 = 0.9
beta2 = 0.999
e = 10 ** -8

Vdw = np.zeros(shape = (len(x_t),1))

Sdw = np.zeros(shape = (len(x_t),1))
alpha = 0.001

#repeat

for i in range(repeat):
    pred_y = np.dot(x, w)
    loss = pred_y - y
    
    if(i % 500 == 0 ):
        cost = np.sum(loss ** 2)/len(loss)
        cost_a = math.sqrt(cost)
        print(cost_a)
    
    grad = 2*np.dot(x_t, loss)
    
    Vdw = beta1 * Vdw + (1 - beta1) * grad
    Sdw = beta2 * Sdw + (1 - beta2) * grad ** 2
    
    Vdw_c = Vdw / (1 - beta1 ** (i+1))
    Sdw_c = Sdw / (1 - beta2 ** (i+1))

    w = w*0.99996 - alpha * Vdw_c / (np.sqrt(Sdw_c) + e)

27.071214829194115
23.505871028325046
22.63761699228748
22.003383006168963
21.44309823398102
20.920347215069853
20.426292461693286
19.95572400054812
19.504280185793046
19.068577836341465
18.646230169783987
18.23566940240077
17.83592034052478
17.446431386738976
17.066948058726854
16.69737562278674
16.337659739826858
15.987718395127764
15.647412668912821
15.31654389478258
14.994868356869391
14.682112582586424
14.377977394977403
14.082135090967915
13.794235520387133
13.513931882918845
13.240918365802985
12.97495822643825
12.715886357836041
12.46358901459685
12.217977090132642
11.978966852337239
11.746470558545886
11.520392899594034
11.300631888874406
11.087080672856237
10.879627167247872
10.678157892504482
10.482556126166614
10.292704599414163
10.108486351756152
9.929785720837613
9.756490778658517
9.58849345905834
9.425690056215974
9.26798247931243
9.115276067822208
8.967479122744502
8.824501497238867
8.686254196354403
8.552647461788167
8.423592638230742
8.299000521634758
8.17878159645519

In [11]:
np.save('weight_adam.npy',w)

In [12]:
np.save('mean.npy',mean)
np.save('std.npy',std)